Exercise 1

In [ ]:
## No, it is not a good idea to try scaling the input features. 

Exercise 2

In [ ]:
## Yes, decreasing the max_depth would be a way to try and fit over-fitting.

Exercise 3

In [ ]:
## g). All of the above

Exercise 4

In [ ]:
## d). a. and c.

Exercise 5

Exercise 6

In [19]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier


## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'mckayla-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to read from s3 bucket
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

##Reading the csv file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [20]:
## Removing the observations with N/As
heart = heart.dropna()

In [21]:
## Defining the input and target variables
X = heart[['male','age', 'education','currentSmoker', 'cigsPerDay', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes','totChol', 'sysBP', 'diaBP','BMI','heartRate', 'glucose']]
Y = heart['TenYearCHD']

for i in range(0, 100):
    ## Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

    ## Building the random forest model
    RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train, Y_train)

    ## Predicting on test
    RF_pred = RF_md.predict_proba(X_test)[:,1]

## Extracting the feature importances
importances_avg = pd.DataFrame({'Feature': X_train.columns, 'Importance': RF_md.feature_importances_}) 
importances_avg = importances_avg.sort_values(by = 'Importance', ascending = False)
importances_avg

,Feature,Importance
1,age,0.197830
10,sysBP,0.171154
11,diaBP,0.126564
14,glucose,0.100155
12,BMI,0.084142
9,totChol,0.073502
7,prevalentHyp,0.059707
4,cigsPerDay,0.043455
13,heartRate,0.042383
2,education,0.032030


In [37]:
## Top 5 variables as predictors
## Defining the input and target variables for Random Forest
X = heart[['age', 'sysBP', 'diaBP','BMI', 'glucose']]
Y = heart['TenYearCHD']

## Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [46]:
## Random Forest Model 1

for i in range(0, 100):
    ## Building the Random Forest model
    RF_md1 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

    ## Predicting on test
    RF_pred1 = RF_md1.predict_proba(X_test)[:, 1]

    ## Changing likelihoods to labels
    RF_labels1 = np.where(RF_pred1 < 0.1, 0, 1)

    ## Computing the recall score
    RF_recall1 = recall_score(Y_test, RF_labels1)

In [47]:
## Computing the average recall score
RF_recall1_avg = np.mean(RF_recall1)
RF_recall1_avg

0.75

In [48]:
## Random Forest Model 2

for i in range(0, 100):
    ## Building the Random Forest model
    RF_md2 = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train, Y_train)

    ## Predicting on test
    RF_pred2 = RF_md2.predict_proba(X_test)[:, 1]

    ## Changing likelihoods to labels
    RF_labels2 = np.where(RF_pred2 < 0.1, 0, 1)

    ## Computing the recall score
    RF_recall2 = recall_score(Y_test, RF_labels2)

In [49]:
## Computing the average recall score
RF_recall2_avg = np.mean(RF_recall2)
RF_recall2_avg

0.7068965517241379

In [52]:
## Random Forest Model 3

for i in range(0, 100):
    ## Building the Random Forest model
    RF_md3 = RandomForestClassifier(n_estimators = 500, max_depth = 7).fit(X_train, Y_train)

    ## Predicting on test
    RF_pred3 = RF_md3.predict_proba(X_test)[:, 1]

    ## Changing likelihoods to labels
    RF_labels3 = np.where(RF_pred3 < 0.1, 0, 1)

    ## Computing the recall score
    RF_recall3 = recall_score(Y_test, RF_labels3)

In [53]:
## Computing the average recall score
RF_recall3_avg = np.mean(RF_recall3)
RF_recall3_avg

0.7068965517241379

In [ ]:
## The best random forest model would be the first, using a maximum depth of 3. The average recall score is 0.75, which would be the best to predict the TenYearCHD variable.